In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas.tseries.offsets import *
from tqdm import tqdm
from functools import reduce
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning, module="numpy")

# CRSP

In [ ]:
crsp = pd.read_parquet('../data/WRDS/crsp_m.parquet')
crsp['prc'] = abs(crsp['prc'])
crsp['ME'] = (crsp['prc']) * crsp['shrout']
crsp.sort_values(by=['permno','YearMonth'], inplace=True)
crsp['bh1m'] = crsp.groupby('permno')['retadj'].shift(-1)
crsp['prc_l1'] = crsp.groupby('permno')['prc'].shift(1)
crsp.duplicated(subset=['permno','YearMonth']).sum()
crsp.rename(columns={'permno':'PERMNO'}, inplace=True)

In [ ]:
crsp.duplicated(subset=['PERMNO','YearMonth']).sum()

# IBES

In [ ]:
### IBES summary actual ###
## CRSP-IBES link table
iclink = pd.read_csv('../data/WRDS/iclink_WRDS.csv')
iclink['sdate'] = pd.to_datetime(iclink['sdate'])
iclink['edate'] = pd.to_datetime(iclink['edate'])
iclink.dropna(subset=['PERMNO'], inplace=True)

# PERMNO -- > GVKEY
ccm = pd.read_parquet('../data/WRDS/ccm.parquet')
# if linkenddt is missing then set to today date
ccm['linkenddt']=ccm['linkenddt'].fillna(pd.to_datetime('today'))
ccm.rename(columns={'permno':'PERMNO'}, inplace=True)
ccm['gvkey'] = ccm['gvkey'].astype(float)

compustat_q = pd.read_csv('../data/WRDS/comp_fundq.csv',
                          usecols=['gvkey','datadate','rdq',
                                   ],
                        #   nrows=20
                         )
compustat_q['PENDS'] = pd.to_datetime(compustat_q['datadate'],format='%Y%m%d')
compustat_q['ANNDATS'] = pd.to_datetime(compustat_q['rdq'], format='%Y%m%d')

# actual earnings from IBES summary unadjusted
AE = pd.read_parquet('../data/WRDS/Actual_EPS_summary_unadjusted_2023.parquet')
AE = AE[(AE['CUSIP']!='') & (AE['CUSIP']!='00000000')]
AE['YearMonth'] = pd.to_datetime(AE['STATPERS']) + MonthEnd(0)

# TICKER --> PERMNO
AE = AE.merge(iclink[['TICKER','PERMNO','NCUSIP','sdate','edate']],
         on=['TICKER'],
        )
AE = AE[(AE['STATPERS']>=AE['sdate']) \
        & (AE['STATPERS']<=AE['edate']) \
        & (AE['CUSIP']==AE['NCUSIP'])
        ]

# PERMNO --> GVKEY
AE = AE.merge(ccm, on=['PERMNO'])
AE = AE[(AE['YearMonth']>=AE['linkdt']) & (AE['YearMonth']<=AE['linkenddt'])]

AE['FY0EDATS'] = pd.to_datetime(AE['FY0EDATS'], format='%Y-%m-%d')
AE['INT0DATS'] = pd.to_datetime(AE['INT0DATS'], format='%Y-%m-%d')

# GVKEY --> rdq (ANNDATS)
AE_A = AE[['TICKER','PERMNO','gvkey','FY0EDATS','FY0A']]\
        .rename(columns={'FY0EDATS':'PENDS','FY0A':'EPS_true'})\
        .drop_duplicates(subset=['TICKER','PENDS'])
AE_A = AE_A.merge(compustat_q[['gvkey','PENDS','ANNDATS']],
           on=['gvkey','PENDS'], how='left'
           )
AE_A['PDICITY'] = 'ANN'

# quarterly actual AE
AE_Q = AE[['TICKER','PERMNO','gvkey','INT0DATS','INT0A']]\
        .rename(columns={'INT0DATS':'PENDS','INT0A':'EPS_true'})\
        .drop_duplicates(subset=['TICKER','PENDS'])
AE_Q = AE_Q.merge(compustat_q[['gvkey','PENDS','ANNDATS']],
              on=['gvkey','PENDS'], how='left'
              )
AE_Q['PDICITY'] = 'QTR'

In [ ]:
AE_summary = pd.concat([AE_A[['TICKER','PENDS','EPS_true','ANNDATS','PDICITY']],
           AE_Q[['TICKER','PENDS','EPS_true','ANNDATS','PDICITY']]
           ], axis=0)

AE_summary.to_parquet('../data/Results/AE_summary.parquet')

In [ ]:
AE_summary = pd.read_parquet('../data/Results/AE_summary.parquet')
## Full IBES actual
EPS_true = pd.read_parquet('../data/WRDS/Actual_EPS_detail_unadjusted_2023.parquet')
EPS_true['EPS_true'] = EPS_true['VALUE']
EPS_true['PENDS'] = pd.to_datetime(EPS_true['PENDS'], format='%Y-%m-%d')
EPS_true['ANNDATS'] = pd.to_datetime(EPS_true['ANNDATS'], format='%Y-%m-%d')
EPS_true = EPS_true[['TICKER', 'PENDS', 'EPS_true', 'ANNDATS', 'PDICITY']].copy()

# V1: combine detail and summary
EPS_true = pd.concat([EPS_true, AE_summary], axis=0)
EPS_true.dropna(subset=['EPS_true','ANNDATS'], inplace=True)
EPS_true.drop_duplicates(subset=['TICKER','PENDS','PDICITY'], inplace=True)

# V2: fill ANADATS by IBES detail actual
# EPS_true = AE_summary.merge(EPS_true[['TICKER','PENDS','PDICITY',
#                                       'ANNDATS','EPS_true'
#                                       ]].rename(columns={'ANNDATS':'ANNDATS_detail',
#                                                          'EPS_true':'EPS_true_detail'
#                                                         }),
#                             on=['TICKER','PENDS','PDICITY'],
#                             how='left'
#                             )
# EPS_true['ANNDATS'] = EPS_true['ANNDATS'].fillna(EPS_true['ANNDATS_detail'])
# EPS_true['EPS_true'] = EPS_true['EPS_true'].fillna(EPS_true['EPS_true_detail'])
# EPS_true.drop(columns=['ANNDATS_detail','EPS_true_detail'], inplace=True)
# EPS_true.dropna(subset=['EPS_true','ANNDATS'], inplace=True)
# EPS_true.drop_duplicates(subset=['TICKER','PENDS','PDICITY'], inplace=True)
# print(AE_summary.shape, EPS_true.shape)

# V3: only use IBES detail actual
# EPS_true = AE_summary.copy()

# V4: only use IBES summary actual
# EPS_true = EPS_true.copy()

In [ ]:
# Last EPS
EPS_true_qtr = EPS_true[EPS_true['PDICITY'] == 'QTR'].sort_values(by=['TICKER','PENDS'])
EPS_true_ann = EPS_true[EPS_true['PDICITY'] == 'ANN'].sort_values(by=['TICKER','PENDS'])

EPS_true_qtr['EPS_true_l1'] = EPS_true_qtr.groupby('TICKER')['EPS_true'].shift(1)
EPS_true_qtr['ANNDATS_l1'] = EPS_true_qtr.groupby('TICKER')['ANNDATS'].shift(1)

EPS_true_ann['EPS_true_l1'] = EPS_true_ann.groupby('TICKER')['EPS_true'].shift(1)
EPS_true_ann['ANNDATS_l1'] = EPS_true_ann.groupby('TICKER')['ANNDATS'].shift(1)

In [ ]:
## IBES consensus
consensus = pd.read_parquet('../data/WRDS/Forecast_EPS_summary_unadjusted_2023.parquet')
consensus['STATPERS'] = pd.to_datetime(consensus['STATPERS'], format='%Y-%m-%d')
consensus['FPEDATS'] = pd.to_datetime(consensus['FPEDATS'], format='%Y-%m-%d')
consensus['YearMonth'] = consensus['STATPERS'] + MonthEnd(0)
consensus['EPS_ana'] = consensus['MEANEST']

# Merge with Actual
consensus_quarter = consensus[consensus.FPI.isin(['6','7','8'])].copy()
consensus_annual = consensus[consensus.FPI.isin(['1','2'])].copy()

consensus_quarter = consensus_quarter.merge(EPS_true_qtr[['TICKER','PENDS','EPS_true','ANNDATS','ANNDATS_l1','EPS_true_l1']],
                                            left_on=['TICKER','FPEDATS'],
                                            right_on=['TICKER','PENDS'],
                                            how='left'
                                           )

consensus_annual = consensus_annual.merge(EPS_true_ann[['TICKER','PENDS','EPS_true','ANNDATS','ANNDATS_l1','EPS_true_l1']],
                                            left_on=['TICKER','FPEDATS'],
                                            right_on=['TICKER','PENDS'],
                                            how='left'
                                         )

consensus = pd.concat([consensus_quarter, consensus_annual], axis=0)

# ICLINK
consensus = consensus.merge(iclink[['TICKER','PERMNO','sdate','edate','NCUSIP']],
                           on=['TICKER'])
consensus = consensus[(consensus['STATPERS']>=consensus['sdate']) & (consensus['STATPERS']<=consensus['edate'])]
consensus = consensus[consensus['CUSIP'] == consensus['NCUSIP']]
consensus = consensus.drop(columns=['sdate','edate','NCUSIP'])

In [ ]:
# Adjusted Actual
def process_consensus(FPI, consensus, crsp):
    df = consensus[consensus['FPI']==FPI][['TICKER','PERMNO','STATPERS','EPS_ana',
                                        'EPS_true','EPS_true_l1','ANNDATS','ANNDATS_l1',
                                        ]].drop_duplicates(subset=['TICKER','STATPERS']).copy()

    # ANN month cfacshr
    df['ANN_m'] = df['ANNDATS'] + MonthEnd(0)
    df = df.merge(crsp[['PERMNO','YearMonth','cfacshr']],
                left_on=['PERMNO','ANN_m'],
                right_on=['PERMNO','YearMonth'],
                )
    df['EPS_true'] = df['EPS_true']/df['cfacshr']
    df.drop(columns=['YearMonth','cfacshr'], inplace=True)

    # Last ANN month cfacshr
    df['ANN_m'] = df['ANNDATS_l1'] + MonthEnd(0)
    df = df.merge(crsp[['PERMNO','YearMonth','cfacshr']],
                left_on=['PERMNO','ANN_m'],
                right_on=['PERMNO','YearMonth'],
                how='left'
                )
    df['EPS_true_l1'] = df['EPS_true_l1']/df['cfacshr']
    df.drop(columns=['YearMonth','cfacshr','ANN_m'], inplace=True)

    df['YearMonth'] = df['STATPERS'] + MonthEnd(0)

    df = df.merge(crsp[['PERMNO','YearMonth','cfacshr']], on=['PERMNO','YearMonth'])

    # Adjust to today cfacshr
    df['EPS_true'] = df['EPS_true'] * df['cfacshr']
    df['EPS_true_l1'] = df['EPS_true_l1'] * df['cfacshr']
    df.drop(columns=['cfacshr'], inplace=True)

    return df

q1 = process_consensus('6', consensus, crsp)
q2 = process_consensus('7', consensus, crsp)
q3 = process_consensus('8', consensus, crsp)
y1 = process_consensus('1', consensus, crsp)
y2 = process_consensus('2', consensus, crsp)

In [ ]:
# Rename columns and Merge all horizon
q1.rename(columns={'EPS_true_l1':'EPS_true_l1_q1',
                   'EPS_true':'EPS_true_q1',
                   'EPS_ana':'EPS_ana_q1',
                   'ANNDATS':'ANNDATS_q1','ANNDATS_l1':'ANNDATS_l1_q1',
                  },
          inplace=True)
q2.rename(columns={'EPS_true_l1':'EPS_true_l1_q2',
                   'EPS_true':'EPS_true_q2',
                   'EPS_ana':'EPS_ana_q2',
                   'ANNDATS':'ANNDATS_q2','ANNDATS_l1':'ANNDATS_l1_q2',
                  },
            inplace=True)
q3.rename(columns={'EPS_true_l1':'EPS_true_l1_q3',
                   'EPS_true':'EPS_true_q3',
                   'EPS_ana':'EPS_ana_q3',
                   'ANNDATS':'ANNDATS_q3','ANNDATS_l1':'ANNDATS_l1_q3',
                  },
            inplace=True)
y1.rename(columns={'EPS_true_l1':'EPS_true_l1_y1',
                   'EPS_true':'EPS_true_y1',
                   'EPS_ana':'EPS_ana_y1',
                   'ANNDATS':'ANNDATS_y1','ANNDATS_l1':'ANNDATS_l1_y1',
                  },
            inplace=True)
y2.rename(columns={'EPS_true_l1':'EPS_true_l1_y2',
                   'EPS_true':'EPS_true_y2',
                   'EPS_ana':'EPS_ana_y2',
                   'ANNDATS':'ANNDATS_y2','ANNDATS_l1':'ANNDATS_l1_y2',
                  },
            inplace=True)

ana_all = reduce(lambda x,y: pd.merge(x,y,
                                       on=['PERMNO','YearMonth'],
                                       how='outer'),
                 [q1.drop(columns=['TICKER']),
                  q2.drop(columns=['TICKER','STATPERS']),
                  q3.drop(columns=['TICKER','STATPERS']),
                  y1.drop(columns=['TICKER','STATPERS']),
                  y2.drop(columns=['TICKER','STATPERS',])])

In [ ]:
df = ana_all.merge(crsp[['PERMNO','YearMonth','siccd',
                         'ret', 'prc', 'bh1m', 'shrout', 'ME','prc_l1',
                        #  'shrcd', 'exchcd'
                        ]],
                   on=['PERMNO','YearMonth'],
                   )

In [ ]:
df.duplicated(subset=['PERMNO','YearMonth']).sum()

# Financial Ratios

In [ ]:
#### Financial Ratios ####
ratios = pd.read_stata('../data/WRDS/financial_ratio.dta')
ratios['public_date'] = ratios['public_date'] + MonthEnd(0)
ratios['gvkey'] = ratios['gvkey'].astype(float)

#### COMUPSTAT ####
compa = pd.read_parquet('../data/WRDS/compa.parquet')
compa['gvkey'] = compa['gvkey'].astype(float)

## SIC code from Compustat
ratios = ratios.merge(compa[['gvkey','datadate','sich']],
                     left_on = ['gvkey','adate'],
                     right_on = ['gvkey','datadate'],
                     how = 'left'
                    )
ratios.drop_duplicates(subset=['permno','public_date'], inplace=True)

df = df.merge(ratios.rename(columns={'permno':'PERMNO','public_date':'YearMonth'}),
              on=['PERMNO','YearMonth'],
              how = 'left'
             )

In [ ]:
df.duplicated(subset=['PERMNO','YearMonth']).sum()

In [ ]:
## FF49 Industry
def zip_2_list(x):
    tmp = []
    for i, j in x:
        tmp += list(range(i,j))
    return tmp

def fama_industry(sic, fama10):
    for i in fama10.index:
        if sic in fama10[i]:
            return i
    ## others
    return 49

# If sich is missing, use siccd from CRSP
df['sic'] = np.where(df['sich'].isna(), df['siccd'], df['sich'])
df['sic'] = df['sic'].astype(int)

fama49 = pd.read_csv('../data/Other/Siccodes49.csv')
fama49 = fama49.groupby('ff49').apply(lambda x: zip_2_list(zip(x.sic1, x.sic2+1)))

_sic = df['sic'].unique()
_sicff = pd.DataFrame(_sic).rename(columns={0:'sic'})
_sicff['fama49'] = _sicff['sic'].apply(lambda x: fama_industry(x,fama49))

df = pd.merge(df, _sicff, how='left', on=['sic'])

In [ ]:
df.duplicated(subset=['PERMNO','YearMonth']).sum()

In [ ]:
## Fill NA with Industry Median
## preprocess
ratio_chars = ['CAPEI', 'bm',
       'evm', 'pe_exi', 'pe_inc', 'ps', 'pcf',
       'dpr', 'npm', 'opmbd', 'opmad', 'gpm', 'ptpm', 'cfm', 'roa', 'roe',
       'roce', 'efftax', 'aftret_eq', 'aftret_invcapx', 'aftret_equity',
       'pretret_noa', 'pretret_earnat', 'GProf', 'equity_invcap',
       'debt_invcap', 'totdebt_invcap', 'capital_ratio', 'int_debt',
       'int_totdebt', 'cash_lt', 'invt_act', 'rect_act', 'debt_at',
       'debt_ebitda', 'short_debt', 'curr_debt', 'lt_debt', 'profit_lct',
       'ocf_lct', 'cash_debt', 'fcf_ocf', 'lt_ppent', 'dltt_be', 'debt_assets',
       'debt_capital', 'de_ratio', 'intcov', 'intcov_ratio', 'cash_ratio',
       'quick_ratio', 'curr_ratio', 'cash_conversion', 'inv_turn', 'at_turn',
       'rect_turn', 'pay_turn', 'sale_invcap', 'sale_equity', 'sale_nwc',
       'rd_sale', 'adv_sale', 'staff_sale', 'accrual', 'ptb', 'PEG_trailing',
       'divyield']

## XX per share characteristics: IN Online Appendix A.2, BHL states that they "consider another twenty-six
# fundamental values per share derived from these financial ratios"
# We recover these features from their persudo-data shared in RFS code & data
# See the data they shared: "/Earnings Forecasts/SampleFigure1.csv". Columns 'BU' to 'CR', totaling 24
# I add "sales_p" & "invcap_p" to make it 26
per_share_chars = ['dividend_p','BE_p','Liability_p','cur_liability_p','LT_debt_p',
                  'cash_p', 'total_asset_p', 'tot_debt_p', 'accrual_p', 'EBIT_p',
                   'cur_asset_p', 'pbda_p', 'ocf_p', 'inventory_p', 'receivables_p',
                   'Cur_debt_p', 'interest_p', 'fcf_ocf_p', 'evm_p',
                   'sales_p', 'invcap_p', 'c_equity_p', 'rd_p', 'opmad_p', 'gpm_p','ptpm_p'
                  ]

df['dividend_p'] = df['divyield'] * df['prc']
df['BE_p'] = df['bm'] * df['prc'] # book-equity
df['Liability_p'] = df['de_ratio'] * df['BE_p'] # Total Debt
df['cur_liability_p'] = df['curr_debt'] * df['Liability_p']
df['LT_debt_p'] = df['lt_debt'] * df['Liability_p']
df['cash_p'] = df['cash_lt'] * df['Liability_p']
df['total_asset_p'] = df['Liability_p'] / df['debt_at']
df['tot_debt_p'] = df['debt_assets'] * df['total_asset_p']
df['accrual_p'] = df['accrual'] * df['total_asset_p']
df['EBIT_p'] = df['debt_ebitda'] / df['tot_debt_p']
df['cur_asset_p'] = df['curr_ratio']*df['cur_liability_p']
df['pbda_p'] = df['profit_lct'] * df['cur_liability_p'] # Operating Income before D&A
df['ocf_p'] = df['ocf_lct'] * df['cur_liability_p'] # Operating Cash Flow
df['inventory_p'] = df['invt_act'] * df['cur_asset_p']
df['receivables_p'] = df['rect_act'] * df['cur_asset_p']
df['Cur_debt_p'] = df['short_debt'] * df['total_asset_p'] # Short-term Debt
df['interest_p'] = df['int_totdebt'] * df['tot_debt_p']
df['fcf_ocf_p'] = df['fcf_ocf'] * df['ocf_p'] # Free Cash Flow
df['evm_p'] = df['evm'] * df['EBIT_p'] # Multiple of Enterprise Value

## ADD by YANDI ##
df['sales_p'] = df['sale_equity'] * df['BE_p'] # Sales
df['invcap_p'] = df['debt_invcap'] / df['LT_debt_p'] # Invested Capital

## Recover theirs
df['c_equity_p'] = df['equity_invcap'] * df['invcap_p'] # Common Equity
df['rd_p'] = df['rd_sale'] * df['sales_p'] # R&D
df['opmad_p'] = df['opmad'] * df['sales_p'] # Operating Income After Depreciation
df['gpm_p'] = df['gpm']  * df['sales_p'] # Gross Profit
df['ptpm_p'] = df['ptpm']  * df['sales_p'] # Pretax Income

df.replace([-np.inf, np.inf], np.nan, inplace=True)

In [ ]:
## Fill NA with Industry Median
for v in tqdm(ratio_chars+per_share_chars):
    df[v] = df.groupby(['YearMonth','fama49'], group_keys=False)[v]\
              .apply(lambda x: x.fillna(x.median()) if not x.isna().all() else x)

## In case some characteristics are all NA in some industry
for v in tqdm(ratio_chars+per_share_chars):
    df[v] = df.groupby(['YearMonth'], group_keys=False)[v].apply(lambda x: x.fillna(x.median()))

# Macro Data

In [ ]:
### Macro Data
RGDP = pd.read_excel('../data/Macro/RGDP.xlsx').set_index('DATE')
RGDP = RGDP.apply(lambda x: np.log(x.dropna()).diff().iloc[-1], axis=0)
RGDP.index = pd.date_range(start='1965-11', end='2024-04', freq='ME')

RCON = pd.read_excel('../data/Macro/RCON.xlsx').set_index('DATE')
RCON = RCON.apply(lambda x: np.log(x.dropna()).diff().iloc[-1], axis=0)
RCON.index = pd.date_range(start='1965-11', end='2024-04', freq='ME')

INDPROD = pd.read_excel('../data/Macro/INDPROD.xlsx').set_index('DATE')
INDPROD = INDPROD.apply(lambda x: np.log(x.dropna()).diff().iloc[-1], axis=0)
INDPROD.index = pd.date_range(start='1962-11', end='2024-03', freq='ME')

UNEMP = pd.read_excel('../data/Macro/UNEMP.xlsx').set_index('DATE')
UNEMP = UNEMP['RUC24Q1'].dropna()
UNEMP.index = pd.date_range(start='1948-01', end='2024-02', freq='ME')
## LAG one month, we can only observe last month UNEMP
UNEMP = UNEMP.shift(1)

macro = pd.DataFrame({'RGDP':RGDP,'RCON':RCON,'INDPROD':INDPROD,'UNEMP':UNEMP})

In [ ]:
df = df.merge(macro, left_on='YearMonth', right_index=True)

In [ ]:
df.duplicated(subset=['PERMNO','YearMonth']).sum()

# Final Data

In [ ]:
ratio_chars = ['CAPEI', 'bm',
       'evm', 'pe_exi', 'pe_inc', 'ps', 'pcf',
       'dpr', 'npm', 'opmbd', 'opmad', 'gpm', 'ptpm', 'cfm', 'roa', 'roe',
       'roce', 'efftax', 'aftret_eq', 'aftret_invcapx', 'aftret_equity',
       'pretret_noa', 'pretret_earnat', 'GProf', 'equity_invcap',
       'debt_invcap', 'totdebt_invcap', 'capital_ratio', 'int_debt',
       'int_totdebt', 'cash_lt', 'invt_act', 'rect_act', 'debt_at',
       'debt_ebitda', 'short_debt', 'curr_debt', 'lt_debt', 'profit_lct',
       'ocf_lct', 'cash_debt', 'fcf_ocf', 'lt_ppent', 'dltt_be', 'debt_assets',
       'debt_capital', 'de_ratio', 'intcov', 'intcov_ratio', 'cash_ratio',
       'quick_ratio', 'curr_ratio', 'cash_conversion', 'inv_turn', 'at_turn',
       'rect_turn', 'pay_turn', 'sale_invcap', 'sale_equity', 'sale_nwc',
       'rd_sale', 'adv_sale', 'staff_sale', 'accrual', 'ptb', 'PEG_trailing',
       'divyield']

per_share_chars = ['dividend_p','BE_p','Liability_p','cur_liability_p','LT_debt_p',
                  'cash_p', 'total_asset_p', 'tot_debt_p', 'accrual_p', 'EBIT_p',
                   'cur_asset_p', 'pbda_p', 'ocf_p', 'inventory_p', 'receivables_p',
                   'Cur_debt_p', 'interest_p', 'fcf_ocf_p', 'evm_p',
                   'sales_p', 'invcap_p', 'c_equity_p', 'rd_p', 'opmad_p', 'gpm_p','ptpm_p'
                  ]

macro_chars = ['RGDP', 'RCON', 'INDPROD', 'UNEMP']

fundamental_chars = ['ret', 'prc',
                    'EPS_true_l1_q1','EPS_true_l1_q2','EPS_true_l1_q3',
                    'EPS_true_l1_y1','EPS_true_l1_y2',
                    ]

analyst_chars = ['EPS_ana_q1','EPS_ana_q2','EPS_ana_q3','EPS_ana_y1','EPS_ana_y2']

targets = ['EPS_true_q1', 'EPS_true_q2', 'EPS_true_q3', 'EPS_true_y1', 'EPS_true_y2']

In [ ]:
### Lag one month information ###
### Except for analyst forecasts
df.sort_values(by=['PERMNO', 'YearMonth'], inplace=True)
vars_lag = ratio_chars + per_share_chars + macro_chars + fundamental_chars
df[vars_lag] = df.groupby('PERMNO')[vars_lag].shift(1)

In [ ]:
# ## FillNA with Industry Median
fillNA = ratio_chars + per_share_chars + fundamental_chars
for v in tqdm(fillNA):
    df[v] = df.groupby(['YearMonth','fama49'], group_keys=False)[v].apply(lambda x: x.fillna(x.median()))
## In case some characteristics are all NA in some industry
for v in tqdm(fillNA + macro_chars):
    df[v] = df.groupby(['YearMonth'], group_keys=False)[v].apply(lambda x: x.fillna(x.median()))

In [ ]:
df_tmp = df[(df['YearMonth'] >= '1984-01-01') & (df['YearMonth'] <= '2019-12-31')].reset_index(drop=True).copy()

In [ ]:
# winsorization period-by-period
cols = ratio_chars + per_share_chars + fundamental_chars + analyst_chars + targets
df_tmp[cols] = df_tmp.groupby('YearMonth',group_keys=False)[cols]\
                             .transform(lambda x: x.clip(x.quantile(0.01),x.quantile(0.99)))

In [ ]:
df_tmp.rename(columns={'PERMNO':'permno'}, inplace=True)

In [ ]:
df_tmp.head()

In [ ]:
# Our working data
df_tmp.to_parquet('../data/Results/df_train_new.parquet')